In [1]:
#!pip install kdtree
#!pip install scikit-learn==1.0.2

In [2]:
#Import
import os
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import urllib
from urllib.request import urlopen, Request
from sklearn.neighbors import NearestNeighbors
from io import BytesIO
from zipfile import ZipFile
import requests
import shapefile
import statsmodels.api as sm
import statsmodels.formula.api as smf
from enum import unique
import pyproj #https://stackoverflow.com/questions/68317672/coordinate-conversion-script-isnt-giving-me-an-accurate-reading-svy21-to-wgs84
import geopy.distance #https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
import sklearn.neighbors
from sklearn.neighbors import BallTree

In [3]:
##Import datasets

#Import resale flat dataset w/ coor
Address=pd.read_csv("Relevant_datasets/Address_Coord.csv",index_col=0)

#Import MRT coords
MRT=pd.read_csv("Relevant_datasets/MRT_coords_B.csv",index_col=0)

#collapse Latitude and Longitude column into one column for both Resale dataset
Address['Coordinates']=Address.apply(lambda x:np.array([x['Lon'],x['Lat']]) ,axis=1)


##global values

#radius of curvature at the equator ≈6378 km
R = 6371008.7714

#credits to :https://www.vibhuagrawal.com/blog/geospatial-nearest-neighbor-search

In [4]:
#Preprocessing

# convert degrees to radians for Address
Address['rLat']= Address.Lat.apply(np.radians)
Address['rLon'] = Address.Lon.apply(np.radians)

# convert degrees to radians for MRT
MRT['rLat']= MRT.Lat.apply(np.radians)
MRT['rLon'] = MRT.Lon.apply(np.radians)

#credits to :https://www.vibhuagrawal.com/blog/geospatial-nearest-neighbor-search

In [5]:
#Balltree
#Balltree is preferred because of the nature of how coordinates work, which makes calculating distance as if coordinates are points on 2D plane a bad idea.
#Balltree allows for haversine distance to be used, which is a better calculation of distance between 2 points
#haversine*r=distance

#Create balltree model, and set haversine distance (important), leaf size>10 would be helpful
Balltree = BallTree(MRT[['rLat','rLon']].values, leaf_size=15, metric='haversine')
def BallTr(POI):
    #Create a numpy array of coordinates for the point of interest and have it be shape(1,2) - used (-1,2) as it is more robust
    POI=np.reshape(np.asarray(POI),(-1,2))
    #generate distance and index 
    dist, ind = Balltree.query(POI, k=1)
    #Get name of MRT station from index
    MRT_STAT=MRT['MRT_Station'][ind[0][0]]
    #return haversine(dist) and MRT station
    return dist, MRT_STAT

#credits to :https://www.vibhuagrawal.com/blog/geospatial-nearest-neighbor-search


In [6]:
#Create empty var: Closest MRT station name
Address['Cl_MRT']=''
#Create empty var: Distance to closest MRT in KM
Address['Distance_MRT_km']=''
for index,row in Address.iterrows():
    #Experimenting with Lat and Lon
    coord=row[['rLat','rLon']].values
    DIST,MRT_Station=BallTr(coord)
    #Distance=haversine*radius
    Distance=DIST[0][0]*R/1000
    Address.loc[index, 'Distance_MRT_km'] = Distance
    Address.loc[index, 'Cl_MRT'] = MRT_Station

In [7]:
#Visual check of a few values 
Address.head(10)

,street_name,Lat,Lon,street_names_oldver,Coordinates,rLat,rLon,Cl_MRT,Distance_MRT_km
0,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,"[103.847235832429, 1.36370657540919]",0.023801,1.812476,ANG MO KIO MRT STATION,0.738894
1,ANG MO KIO AVE 3,1.369306,103.856036,ANG MO KIO AVE 3,"[103.856036447479, 1.36930582535665]",0.023899,1.812630,ANG MO KIO MRT STATION,0.723523
2,ANG MO KIO AVE 4,1.376480,103.839458,ANG MO KIO AVE 4,"[103.839458024511, 1.37647951934552]",0.024024,1.812340,MAYFLOWER MRT STATION,0.643707
3,ANG MO KIO AVE 10,1.366770,103.856590,ANG MO KIO AVE 10,"[103.856589829581, 1.36677040270637]",0.023855,1.812639,ANG MO KIO MRT STATION,0.857127
4,ANG MO KIO AVE 5,1.377311,103.860486,ANG MO KIO AVE 5,"[103.860485872465, 1.37731149652799]",0.024039,1.812707,ANG MO KIO MRT STATION,1.465888
5,ANG MO KIO AVE 8,1.370658,103.848803,ANG MO KIO AVE 8,"[103.84880268504, 1.37065826792202]",0.023922,1.812504,ANG MO KIO MRT STATION,0.116439
6,ANG MO KIO AVE 6,1.376194,103.844834,ANG MO KIO AVE 6,"[103.844833519236, 1.37619415250095]",0.024019,1.812434,YIO CHU KANG MRT STATION,0.591043
7,ANG MO KIO AVE 9,1.384029,103.843544,ANG MO KIO AVE 9,"[103.843544330163, 1.38402890348294]",0.024156,1.812412,YIO CHU KANG MRT STATION,0.334415
8,ANG MO KIO AVE 2,1.372855,103.833105,ANG MO KIO AVE 2,"[103.833104628058, 1.37285539502998]",0.023961,1.812230,MAYFLOWER MRT STATION,0.414933
9,BEDOK RESERVOIR RD,1.337019,103.921577,BEDOK RESERVOIR RD,"[103.921576589266, 1.33701921598054]",0.023335,1.813774,BEDOK NORTH MRT STATION,0.473404


In [8]:
Address.to_csv('Relevant_datasets/1.3_Address_MRT.csv')